In [1]:
import sys
from pathlib import Path
import numpy as np

current_dir = Path.cwd()
if str(current_dir) not in sys.path:
    sys.path.insert(0, str(current_dir))

from src import (
    DataConfig, DataLoader, ModelingStrategy, ReleaseManager, BenchmarkPipeline, create_config
)

np.random.seed(42)

In [2]:
data_config = DataConfig(
    mapping_path = 'data/feature_mapping_train.pkl',
    features_path = 'data/processed/train_data_features.feather',
    target_path = 'data/train_data_target.feather'
)

sku_tuples=[(81054, 1335),(81054, 1334),(80558, 1331)]
pipeline = BenchmarkPipeline(data_config)

In [ ]:
data_loader = DataLoader(data_config)
dataset = data_loader.prepare_modeling_dataset(
    sku_tuples=[(81054, 1335),(81054, 1334),(80558, 1331)],
    modeling_strategy=ModelingStrategy.INDIVIDUAL
)

In [3]:
results_lightning_std = pipeline.run_experiment(
    sku_tuples=sku_tuples,
    modeling_strategy=ModelingStrategy.INDIVIDUAL,
    model_type="lightning_standard",
    hyperparameters={
        "hidden_size": 128,
        "lr": 0.001,
        "num_layers": 2,
        "dropout": 0.2,
        "max_epochs": 50,
        "batch_size": 64,
        "random_state": 42
    },
    experiment_name="lightning_standard_test",
    evaluate_on_test=True
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/miniconda3/envs/ml_env/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 25.2 K | train
---------------------------------------------
25.2 K    Trainable params
0         Non-trainable params
25.2 K    Total params
0.101     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode
/opt/miniconda3/envs/ml_env/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
`Train

In [4]:
sample_result = results_lightning_std.training_results[0]
print(f"Model type: {sample_result.model_type}")
print(f"Strategy: {sample_result.modeling_strategy.value}")
print(f"SKU tuples: {sample_result.sku_tuples}")
print(f"Quantile level: {sample_result.quantile_level}")
if sample_result.performance_metrics:
    print(f"RMSE: {sample_result.performance_metrics.get('rmse', 'N/A')}")

Model type: lightning_standard
Strategy: individual
SKU tuples: [(81054, 1335)]
Quantile level: None
RMSE: 17.759819615484293


In [6]:
release_manager = ReleaseManager()
output_dir = Path("./lightning_model_releases")
release_path = release_manager.create_complete_release(
    experiment_results=results_lightning_std,  # Your ExperimentResults from pipeline
    base_output_dir=output_dir
  )



AttributeError: 'ExperimentResults' object has no attribute 'configurations'

In [7]:
results_lightning_std.configurations.get('data_config')

AttributeError: 'ExperimentResults' object has no attribute 'configurations'